In [56]:
import numpy as np 
import pandas as pd
import plotly.graph_objects as go

Data File: https://drive.google.com/file/d/1szmxlcGeROzdwtpB9uHiYbyJZEYt9Qvy/view?usp=sharing
Upload in next code block

In [57]:
from google.colab import files
files = files.upload()

Saving treehacks9.csv to treehacks9 (2).csv


In [58]:
vehicles = pd.read_csv('treehacks9.csv')


In [59]:
vehicles

,Model,Displ,Cyl,Trans,Drive,Fuel,Veh Class,Air Pollution Score,City MPG,Hwy MPG,Cmb MPG,Greenhouse Gas Score,Comb CO2,MSRP
0,HYUNDAI Accent,1.6,4.0,Man-6,2WD,Gasoline,small car,3,29,39,33,7,273,14995
1,HYUNDAI Accent,1.6,4.0,SCV-1,2WD,Gasoline,small car,3,33,41,36,8,248,14995
2,KIA Rio,1.6,4.0,SCV-1,2WD,Gasoline,small car,3,33,41,36,8,249,15395
3,NISSAN Versa,1.6,4.0,CVT,2WD,Gasoline,small car,7,32,40,35,7,254,15720
4,NISSAN Versa,1.6,4.0,Man-5,2WD,Gasoline,small car,7,27,35,30,6,296,15720
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095,LAMBORGHINI Aventador Roadster,6.5,12.0,AMS-7,4WD,Gasoline,small car,1,9,15,10,1,839,399500
1096,FORD GT,3.5,6.0,AutoMan-7,2WD,Gasoline,small car,3,12,18,14,1,632,450000
1097,ROLLS-ROYCE Phantom,6.7,12.0,SemiAuto-8,2WD,Gasoline,large car,3,12,20,14,1,615,492425
1098,ROLLS-ROYCE Phantom EWB,6.7,12.0,SemiAuto-8,2WD,Gasoline,large car,3,12,20,14,1,615,492425


In [60]:
# creates a column for the sustainability score to be inputted
# the sustainability score is an equally weighted combo of the Air Pollution Score, the GG Score, and Comb CO2
vehicles["Sustainability Score"] = pd.Series(dtype=float)
vehicles

,Model,Displ,Cyl,Trans,Drive,Fuel,Veh Class,Air Pollution Score,City MPG,Hwy MPG,Cmb MPG,Greenhouse Gas Score,Comb CO2,MSRP,Sustainability Score
0,HYUNDAI Accent,1.6,4.0,Man-6,2WD,Gasoline,small car,3,29,39,33,7,273,14995,NaN
1,HYUNDAI Accent,1.6,4.0,SCV-1,2WD,Gasoline,small car,3,33,41,36,8,248,14995,NaN
2,KIA Rio,1.6,4.0,SCV-1,2WD,Gasoline,small car,3,33,41,36,8,249,15395,NaN
3,NISSAN Versa,1.6,4.0,CVT,2WD,Gasoline,small car,7,32,40,35,7,254,15720,NaN
4,NISSAN Versa,1.6,4.0,Man-5,2WD,Gasoline,small car,7,27,35,30,6,296,15720,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095,LAMBORGHINI Aventador Roadster,6.5,12.0,AMS-7,4WD,Gasoline,small car,1,9,15,10,1,839,399500,NaN
1096,FORD GT,3.5,6.0,AutoMan-7,2WD,Gasoline,small car,3,12,18,14,1,632,450000,NaN
1097,ROLLS-ROYCE Phantom,6.7,12.0,SemiAuto-8,2WD,Gasoline,large car,3,12,20,14,1,615,492425,NaN
1098,ROLLS-ROYCE Phantom EWB,6.7,12.0,SemiAuto-8,2WD,Gasoline,large car,3,12,20,14,1,615,492425,NaN


In [61]:
# data cleaning: erases the '-' in transmission types (for user efficiency) so that the user doesn't have to 
# scroll through a bunch of types of auto/ manual transmissions 
# also makes transmission types more readable ('Man' --> 'Manual')
for i in range(len(vehicles)):
  s = vehicles.iloc[i,3]
  if ('-' in s):
    m = s.index('-')
    l = s[:m]
    vehicles.iloc[i,3] = l

  v = vehicles.iloc[i,10]
  if ('/' in v):
    m = v.index('/')
    l = v[m+1:]
    vehicles.iloc[i,10] = l
    
  if 'AutoMan' in s:
    vehicles.iloc[i,3] = 'AutoManual'
  elif 'Man' in s:
    vehicles.iloc[i,3] = 'Manual'
  if 'Gasoline/Electricity' in vehicles.iloc[i,5]:
    vehicles.iloc[i,5] = 'Hybrid'
  elif 'Electricity' in vehicles.iloc[i,5]:
    vehicles.iloc[i,5] = 'Electric'

  car_class = vehicles.iloc[i, 6]
  #vehicles.iloc[i, 6] = car_class[0].upper() + car_class[1:]
  
  car_class_arr = car_class.split()
  capitalized = ""
  for word in car_class_arr:
    capitalized += word[0].upper() + word[1:]
    capitalized += " "
  vehicles.iloc[i, 6] = capitalized.strip()

  vehicles.iloc[i,13] = float(vehicles.iloc[i,13])
  
  

vehicles
  


,Model,Displ,Cyl,Trans,Drive,Fuel,Veh Class,Air Pollution Score,City MPG,Hwy MPG,Cmb MPG,Greenhouse Gas Score,Comb CO2,MSRP,Sustainability Score
0,HYUNDAI Accent,1.6,4.0,Manual,2WD,Gasoline,Small Car,3,29,39,33,7,273,14995,NaN
1,HYUNDAI Accent,1.6,4.0,SCV,2WD,Gasoline,Small Car,3,33,41,36,8,248,14995,NaN
2,KIA Rio,1.6,4.0,SCV,2WD,Gasoline,Small Car,3,33,41,36,8,249,15395,NaN
3,NISSAN Versa,1.6,4.0,CVT,2WD,Gasoline,Small Car,7,32,40,35,7,254,15720,NaN
4,NISSAN Versa,1.6,4.0,Manual,2WD,Gasoline,Small Car,7,27,35,30,6,296,15720,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095,LAMBORGHINI Aventador Roadster,6.5,12.0,AMS,4WD,Gasoline,Small Car,1,9,15,10,1,839,399500,NaN
1096,FORD GT,3.5,6.0,AutoManual,2WD,Gasoline,Small Car,3,12,18,14,1,632,450000,NaN
1097,ROLLS-ROYCE Phantom,6.7,12.0,SemiAuto,2WD,Gasoline,Large Car,3,12,20,14,1,615,492425,NaN
1098,ROLLS-ROYCE Phantom EWB,6.7,12.0,SemiAuto,2WD,Gasoline,Large Car,3,12,20,14,1,615,492425,NaN


In [62]:
for i in range(len(vehicles)):
  v = vehicles.iloc[i,10]
  if ('/' in v):
    m = v.index('/')
    l = float(v[m+1:])
    vehicles.iloc[i,10] = l
vehicles

,Model,Displ,Cyl,Trans,Drive,Fuel,Veh Class,Air Pollution Score,City MPG,Hwy MPG,Cmb MPG,Greenhouse Gas Score,Comb CO2,MSRP,Sustainability Score
0,HYUNDAI Accent,1.6,4.0,Manual,2WD,Gasoline,Small Car,3,29,39,33,7,273,14995,NaN
1,HYUNDAI Accent,1.6,4.0,SCV,2WD,Gasoline,Small Car,3,33,41,36,8,248,14995,NaN
2,KIA Rio,1.6,4.0,SCV,2WD,Gasoline,Small Car,3,33,41,36,8,249,15395,NaN
3,NISSAN Versa,1.6,4.0,CVT,2WD,Gasoline,Small Car,7,32,40,35,7,254,15720,NaN
4,NISSAN Versa,1.6,4.0,Manual,2WD,Gasoline,Small Car,7,27,35,30,6,296,15720,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095,LAMBORGHINI Aventador Roadster,6.5,12.0,AMS,4WD,Gasoline,Small Car,1,9,15,10,1,839,399500,NaN
1096,FORD GT,3.5,6.0,AutoManual,2WD,Gasoline,Small Car,3,12,18,14,1,632,450000,NaN
1097,ROLLS-ROYCE Phantom,6.7,12.0,SemiAuto,2WD,Gasoline,Large Car,3,12,20,14,1,615,492425,NaN
1098,ROLLS-ROYCE Phantom EWB,6.7,12.0,SemiAuto,2WD,Gasoline,Large Car,3,12,20,14,1,615,492425,NaN


In [63]:
# calculates the max CO2 emission 

#vehicles['Comb CO2'].max()
#column = vehicles['Comb CO2']

max = 0.0
for i in range(len(vehicles)):
  converted_num = float(vehicles.iloc[i,12])
  if converted_num > max:
    max = converted_num
  
max

852.0

In [64]:
max_msrp = 0.0
for i in range(len(vehicles)):
  msrp_num = float(vehicles.iloc[i,13])
  if msrp_num > max_msrp:
    max_msrp = msrp_num
max_msrp

625000.0

In [65]:
vehicles = vehicles.rename(columns={'Veh Class': 'Veh_Class'})

In [66]:
# creates a list of all the uniques types per filter

transmission_types = sorted(vehicles.Trans.unique())
drivetrain = sorted(vehicles.Drive.unique())
vehicle_size = sorted(vehicles['Veh_Class'].unique())
fuel_type = sorted(vehicles['Fuel'].unique())

print(transmission_types)
print(drivetrain)
print(vehicle_size)
print(fuel_type)

['AMS', 'Auto', 'AutoManual', 'CVT', 'Manual', 'SCV', 'SemiAuto']
['2WD', '4WD']
['Large Car', 'Midsize Car', 'Minivan', 'Pickup', 'Small Car', 'Small SUV', 'Special Purpose', 'Standard SUV', 'Station Wagon']
['Diesel', 'Electric', 'Gasoline', 'Hybrid', 'Hydrogen']


In [67]:
for i in range(len(vehicles)):
  air_score = float(vehicles.iloc[i,7])/10
  gg_score = (float(vehicles.iloc[i,11]))/10
  co2 = 1 - float(vehicles.iloc[i,12])/max
  vehicles.iloc[i,14] = round(10 * (air_score + gg_score + co2)/3,2)
vehicles

,Model,Displ,Cyl,Trans,Drive,Fuel,Veh_Class,Air Pollution Score,City MPG,Hwy MPG,Cmb MPG,Greenhouse Gas Score,Comb CO2,MSRP,Sustainability Score
0,HYUNDAI Accent,1.6,4.0,Manual,2WD,Gasoline,Small Car,3,29,39,33,7,273,14995,5.60
1,HYUNDAI Accent,1.6,4.0,SCV,2WD,Gasoline,Small Car,3,33,41,36,8,248,14995,6.03
2,KIA Rio,1.6,4.0,SCV,2WD,Gasoline,Small Car,3,33,41,36,8,249,15395,6.03
3,NISSAN Versa,1.6,4.0,CVT,2WD,Gasoline,Small Car,7,32,40,35,7,254,15720,7.01
4,NISSAN Versa,1.6,4.0,Manual,2WD,Gasoline,Small Car,7,27,35,30,6,296,15720,6.51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095,LAMBORGHINI Aventador Roadster,6.5,12.0,AMS,4WD,Gasoline,Small Car,1,9,15,10,1,839,399500,0.72
1096,FORD GT,3.5,6.0,AutoManual,2WD,Gasoline,Small Car,3,12,18,14,1,632,450000,2.19
1097,ROLLS-ROYCE Phantom,6.7,12.0,SemiAuto,2WD,Gasoline,Large Car,3,12,20,14,1,615,492425,2.26
1098,ROLLS-ROYCE Phantom EWB,6.7,12.0,SemiAuto,2WD,Gasoline,Large Car,3,12,20,14,1,615,492425,2.26


In [68]:
!pip install pyngrok
!pip install jupyter-dash

In [69]:
drives = ['2WD']
fuel = ['Gasoline']
size = ['Small Car']
transmissions = ['Auto'] 
filtered_vehicles = vehicles[vehicles.Drive.isin(drives) & vehicles.Veh_Class.isin(size) &
           vehicles.Fuel.isin(fuel) & vehicles.Trans.isin(transmissions)]
low, high = [0,100000]

price_filtered = filtered_vehicles[(filtered_vehicles['MSRP'] >= low) & (filtered_vehicles['MSRP'] <= high)]
price_filtered = price_filtered.sort_values(by=['Sustainability Score'], ascending=False)
price_filtered = price_filtered.reset_index(drop=True)

price_filtered



,Model,Displ,Cyl,Trans,Drive,Fuel,Veh_Class,Air Pollution Score,City MPG,Hwy MPG,Cmb MPG,Greenhouse Gas Score,Comb CO2,MSRP,Sustainability Score
0,MERCEDES-BENZ C300,2.0,4.0,Auto,2WD,Gasoline,Small Car,6,23,35,27,6,324,74850,6.07
1,MERCEDES-BENZ C300,2.0,4.0,Auto,2WD,Gasoline,Small Car,6,24,33,27,6,330,74850,6.04
2,MERCEDES-BENZ E450,3.0,6.0,Auto,2WD,Gasoline,Small Car,6,23,31,26,5,341,69100,5.67
3,MERCEDES-BENZ C300,2.0,4.0,Auto,2WD,Gasoline,Small Car,6,23,31,26,5,347,74850,5.64
4,MERCEDES-BENZ CLS450,3.0,6.0,Auto,2WD,Gasoline,Small Car,6,23,30,26,5,347,74850,5.64
5,MERCEDES-BENZ E450 Convertible,3.0,6.0,Auto,2WD,Gasoline,Small Car,6,23,30,25,5,349,69100,5.63
6,MERCEDES-BENZ S560,4.0,8.0,Auto,2WD,Gasoline,Small Car,5,16,26,20,4,454,72600,4.56


In [70]:
#Load 
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

In [71]:
avg_score = 5
fig = px.scatter(vehicles,
  x = "Cmb MPG",
  y = "Sustainability Score",
  hover_data = ['Model', 'MSRP'], 
  title = "Registered Cars: Sustainability Scores and MPG",
  color = "Sustainability Score",
  color_continuous_scale = px.colors.diverging.RdYlGn,
  opacity = 1,
  color_continuous_midpoint = avg_score, 
  labels=dict(x = "Combined MPG", y = "Sustainability Score"))

fig.update_yaxes(tickvals = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
fig.update_xaxes(title_font=dict(size=22), title_text = "Combined MPG")
fig.update_yaxes(title_font=dict(size=22))
fig.update_layout(
    title={
        'y': 0.935,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    }
)
fig.update_layout(coloraxis_colorbar=dict(
    title=""
))
fig.update_traces(marker=dict(size=10,
                              line=dict(width=1,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'))
fig.update_layout(
    autosize=True,
    height=800,
    margin=dict(l=20, r=20, t=110, b=20),
    paper_bgcolor="LightSteelBlue",
    title=dict(
        font=dict(
            size=36
        )
    ),
    font=dict(
        size=18
    )
)


fig.show()

In [72]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css'] #publicly sourced, not our work
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

app.title = 'eco-drive'
app.layout = html.Div(style={'whiteSpace': 'pre-line','text-align':'center', 'width':'90%', 'margin-left':'5%'}, children = [
  html.Div(className="title", children=[
      html.H1("eco-drive"),
      # have an image logo
      html.Img(src='https://i.ibb.co/zZqRNV7/eco-drive.jpg'),
    html.Br(),
    html.Div(className="section row", children=[
      html.Div(className="float-left col-md-4", children=[
        html.H4("Mission"),
        html.P("We are a group of four Stanford undergrads passionate about climate justice and motivated to change the world's energy regime. The transportation sector is a major contributor to climate change, and with the rise of E-cars surging the markets, we want to incorporate sustainably-oriented thinking in consumers' car buying processes.")
      ]),             
      html.Div(className="float-left col-md-4", children=[
        html.H4("What is eco-drive?"),
        html.P("eco-drive is a consumer-interactive tool that, given a consumer's parameters (budget, vehicle size, fuel type), recommends to them the most sustainable cars. It also functions as a catalogue that ranks cars based off a Sustainability Score, which we calculated based on Air Pollution, Greenhouse Gas, and CO2 Combustion Data given by the EPA.")
      ]),   
    ]),  
    # insert graph here
    html.Div(children=[
        dcc.Tabs(id='tabs-example', children=[
        dcc.Tab(label='All Vehicles', value='tab-1', children=[
            dcc.Graph(id='all_vehicles_graph', style={'height':'55em'}, figure=fig)                                            
        ]),
        dcc.Tab(label='Choose Filters', children=[
            html.Table(style={'display':'90%'}, children=[
                html.Thead([
                    html.Tr([
                        html.Th('Drivetrain'),
                        html.Th('Vehicle Size'),
                        html.Th('Fuel'),
                        html.Th('Transmission')
                    ])
                ]),
                html.Tbody([
                    html.Tr([
                          html.Td(style={'width':'100em'},children=[ dcc.Dropdown(
                                id='drivetrain', value='',
                                options=[{'label': i, 'value': i} for i in drivetrain],
                          ) ]),
                          html.Td(style={'width':'100em'},children=[ dcc.Dropdown(
                                id='vehicle_size', value='',
                                options=[{'label': i, 'value': i} for i in vehicle_size],
                          ) ]),
                          html.Td(style={'width':'100em'},children=[ dcc.Dropdown(
                                id='fuel', value='',
                                options=[{'label': i, 'value': i} for i in fuel_type],
                          ) ]),
                          html.Td(style={'width':'100em'},children=[ dcc.Dropdown(
                                id='transmission', value='',
                                options=[{'label': i, 'value': i} for i in transmission_types],
                          ) ]),     
                    ]),
                ]),
            ]),
            html.Br(),
            html.Div(style={'font-size':'22'},children=[
              dcc.RangeSlider(
                    id='price_range',
                    min=0, max=max_msrp, step=1000,
                    marks={0: '0', 312500:'312,500', 625000: '625,000'},
                    value=[0,625000]
              ),
            ]),
            html.Div(id='price_range_text', style={'font-size':'22px'}),
            html.Br(),
            html.Div(id='results', style={'whiteSpace': 'pre-line','text-align':'center', 'font-size':'22px'}),
            html.Br(),
            dcc.Graph(id='select_graph', style={'height':'55em'}),                                           
          ]),
      ]),         
    ]), #close Tabs Div
    html.Div(children=[
      html.Img(src='https://i.ibb.co/YDP6gLn/eco-drive-1.jpg')           
    ]) #Explanation + How to Use Text Div
  ]) 
])

@app.callback(
    Output('price_range_text', 'children'),
    [Input('price_range', 'value')])
def update_output(value):
    return 'Current Price Range: $' + f"{value[0]:,d}" + " to $" + f"{value[1]:,d}"

@app.callback(
    Output('results', 'children'),
    [Input('drivetrain', 'value'),
     Input('vehicle_size', 'value'),
     Input('fuel', 'value'),
     Input('transmission', 'value'),
     Input('price_range', 'value')]
)
def update_vehicles(drivetrain, vehicle_size, fuel, transmission,price_range):
  string = ' | '
  if len(drivetrain) > 0 and len(vehicle_size) > 0 and len(fuel) > 0 and len(transmission) > 0:
    result = optimize_vehicles(drivetrain, vehicle_size, fuel, transmission, price_range)
    if len(result) > 5:
      result = result.drop(result.index[range(5,len(result))])
    for i in range(len(result)):
        string += str(i+1) + ". " + result.iloc[i,0] + " - Score: " + str(result.iloc[i,14]) + " | "
    if len(result) == 0:
      string = "There are no cars that fit your specifications"
  else:
    string = "Please make a choice for each of the specifications"
  return string

def optimize_vehicles (drivetrain, vehicle_size, fuel, transmission, price_range):
  drives = [drivetrain]
  size = [vehicle_size]
  fuel = [fuel]
  transmissions = [transmission]
  filtered_vehicles = vehicles[vehicles.Drive.isin(drives) & vehicles.Veh_Class.isin(size) &
           vehicles.Fuel.isin(fuel) & vehicles.Trans.isin(transmissions)]

  low, high = price_range

  price_filtered = filtered_vehicles[(filtered_vehicles['MSRP'] >= low) & (filtered_vehicles['MSRP'] <= high)]
  price_filtered = price_filtered.sort_values(by=['Sustainability Score'], ascending=False)
  price_filtered = price_filtered.reset_index(drop=True)
  return price_filtered

@app.callback(
    Output('select_graph', 'figure'),
    [Input('drivetrain', 'value'),
     Input('vehicle_size', 'value'),
     Input('fuel', 'value'),
     Input('transmission', 'value'),
     Input('price_range','value')]
)
def filtered_car_graph(drivetrain, vehicle_size, fuel, transmission, price_range):
  filtered = optimize_vehicles(drivetrain, vehicle_size, fuel, transmission, price_range)
  
  figFiltered = px.scatter(filtered,
  x = "Cmb MPG",
  y = "Sustainability Score",
  hover_data = ['Model','MSRP'],
  title = "Registered Cars: Sustainability Scores and MPG",
  color = "Sustainability Score",
  color_continuous_scale = px.colors.diverging.RdYlGn,
  opacity = 1,
  color_continuous_midpoint = avg_score, 
  labels=dict(x = "Combined MPG", y = "Sustainability Score"))
  figFiltered.update_traces(marker=dict(size=20,
                              line=dict(width=2,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'))
  figFiltered.update_xaxes(title_font=dict(size=22), title_text = "Combined MPG")
  figFiltered.update_yaxes(title_font=dict(size=22), title_text = "Sustainability Score")
  figFiltered.update_layout(
      title={
          'y': 0.935,
          'x': 0.5,
          'xanchor': 'center',
          'yanchor': 'top'
      }
  )
  figFiltered.update_layout(
      coloraxis_colorbar=dict(title=""),
  )
  figFiltered.update_layout(
      autosize=True,
      height=800,
      margin=dict(l=20, r=20, t=110, b=20),
      paper_bgcolor="LightSteelBlue",
      title=dict(
          font=dict(
              size=36
          )
      ),
      font=dict(
          size=18
      ),
      yaxis=dict(nticks=20,range=[0,10.5]),
      xaxis=dict(nticks=14,range=[0,160])
  )

  return figFiltered

app.run_server(mode='external', port=8500)                              

Dash app running on:


<IPython.core.display.Javascript object>

In [73]:
!ngrok authtoken 1oT4twOAUl5hesp2y1VBY0miCz2_37BpvF9WJf2rst26ZFmE9

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [77]:
from pyngrok import ngrok

public_url = ngrok.connect(addr = '8500')
public_url

<NgrokTunnel: "http://5b1ac8c494f1.ngrok.io" -> "http://localhost:8500">

In [78]:
ngrok.kill()